In [ ]:
!pip install --upgrade huggingface_hub[hf_xet] hf_xet peft diffusers transformers accelerate

In [ ]:
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_XET_DOWNLOAD=1
%env HF_XET_HIGH_PERFORMANCE=1

In [ ]:
# ComfyUI setup
%cd  /root/char
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /root/char/ComfyUI
!pip install -r requirements.txt
!mkdir -p /root/char/ComfyUI/custom_nodes
%cd /root/char/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git

%cd /root/char/ComfyUI/models/vae
!curl -LOz "https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors"

%cd /root/char/ComfyUI/models/text_encoders
!curl -LOz "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors"
# !curl -LOz "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors"
!curl -LOz "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors"

%cd /root/char/ComfyUI/models/diffusion_models
!curl -LOz "https://huggingface.co/Comfy-Org/flux1-kontext-dev_ComfyUI/resolve/main/split_files/diffusion_models/flux1-dev-kontext_fp8_scaled.safetensors"
# !curl -LOz "https://huggingface.co/black-forest-labs/FLUX.1-Kontext-dev/resolve/main/flux1-kontext-dev.safetensors"


# !huggingface-cli download black-forest-labs/FLUX.1-dev flux1-dev.safetensors --local-dir /root/char/ComfyUI/models/unet
# !huggingface-cli download black-forest-labs/FLUX.1-Fill-dev flux1-fill-dev.safetensors --local-dir /root/char/ComfyUI/models/unet

# !huggingface-cli download black-forest-labs/FLUX.1-dev ae.safetensors --local-dir /root/char/ComfyUI/models/vae
# !huggingface-cli download comfyanonymous/flux_text_encoders clip_l.safetensors --local-dir /root/char/ComfyUI/models/clip
# !huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp16.safetensors --local-dir /root/char/ComfyUI/models/clip

# %cd /root/char/ComfyUI/models/loras
# !huggingface-cli download kudzueye/Boreal boreal-flux-dev-lora-v04_1000_steps.safetensors --local-dir /root/char/ComfyUI/models/loras
# !huggingface-cli download Shakker-Labs/FLUX.1-dev-LoRA-AntiBlur FLUX-dev-lora-AntiBlur.safetensors --local-dir /root/char/ComfyUI/models/loras
# !huggingface-cli download Shakker-Labs/AWPortrait-FL AWPortrait-FL-lora.safetensors --local-dir /root/char/ComfyUI/models/loras
# %cd /root/char/ComfyUI

# sudo mkdir -p --mode=0755 /usr/share/keyrings
# curl -fsSL https://pkg.cloudflare.com/cloudflare-main.gpg | sudo tee /usr/share/keyrings/cloudflare-main.gpg >/dev/null
# echo 'deb [signed-by=/usr/share/keyrings/cloudflare-main.gpg] https://pkg.cloudflare.com/cloudflared any main' | sudo tee /etc/apt/sources.list.d/cloudflared.list
# sudo apt-get update && sudo apt-get install cloudflared
# cloudflared tunnel --protocol http2 --url http://localhost:8000 # on some machine gotta remove "--protocol http2"

%cd /root/char
!python3 ComfyUI/main.py --port 8000

In [ ]:
import torch
from diffusers import FluxKontextPipeline
from diffusers.utils import load_image
import time

pipe = FluxKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
).to("cuda")

In [ ]:
pipe.load_lora_weights("kudzueye/Boreal", weight_name="boreal-flux-dev-lora-v04_1000_steps.safetensors", adapter_name="boreal")
pipe.set_adapters(["boreal"], [0.7])
pipe.fuse_lora()

In [ ]:
input_image = load_image("https://i.pinimg.com/1200x/30/c7/c0/30c7c096959245ce68c47faa4302c4bf.jpg")

generator = torch.Generator(device="cuda").manual_seed(0)

guidance = 2.5
steps = 40

image = pipe(
    image=input_image,
    prompt="Make a close up photo of her on a neutral background looking straight at the camera wearing a white tank top",
    guidance_scale=guidance,
    num_inference_steps=steps,
    generator=generator,
    width=944,
    height=1104,
).images[0]

image.save(f"out/{guidance}_{steps}.jpg")